# Building your own algorithm container

With Amazon SageMaker, you can package your own algorithms that can than be trained and deployed in the SageMaker environment. This notebook will guide you through an example that shows you how to build a Docker container for SageMaker and use it for training and inference.

In [ ]:
%%time
%%sh

# The name of our algorithm
algorithm_name=decision-trees

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
# S3 prefix
prefix = 'tflearn-planesnet'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
!unzip data/planesnet.json.zip -d data

In [ ]:
%%time
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
%%time
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/decision-trees:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c5.9xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

In [ ]:
%%time
predictor = tree.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
!pip install hickle msgpack argparse>=1.1

In [ ]:
%%time
import sys
import os
import numpy as np
from PIL import Image
from scipy import ndimage

import hickle

in_fname='scene_1.png'
out_fname=None

# Load trained model
# model.load(model_fname)

# Read input image data
im = Image.open(in_fname)
arr = np.array(im)[:,:,0:3]
shape = arr.shape

# Set output fname
if not out_fname: 
    out_fname = os.path.splitext(in_fname)[0] + '_detection.png'

# Create detection variables
detections = np.zeros((shape[0], shape[1]), dtype='uint8')
output = np.copy(arr)

# Sliding window parameters
step = 2
win = 20

# Loop through pixel positions
print('Processing...')
for i in range(0, shape[0]-win, step):
    print('row %1.0f of %1.0f' % (i, (shape[0]-win-1)))

    for j in range(0, shape[1]-win, step):

        # Extract sub chip
        chip = arr[i:i+win,j:j+win,:]
        hickle.dump( chip, '/tmp/chip-%i.hkl' %i )
        
            
        # Predict chip label
        # prediction = model.predict_label([chip / 255.])[0][0]
        print(predictor.predict('/tmp/chip-%i.hkl' %i))

        # Record positive detections
        # if prediction == 1:
        #    detections[i+int(win/2), j+int(win/2)] = 1

# Process detection locations
dilation = ndimage.binary_dilation(detections, structure=np.ones((3,3)))
labels, n_labels = ndimage.label(dilation)
center_mass = ndimage.center_of_mass(dilation, labels, np.arange(n_labels)+1)

# Loop through detection locations
if type(center_mass) == tuple: center_mass = [center_mass]
for i, j in center_mass:
    i = int(i - win/2)
    j = int(j - win/2)
        
# Draw bounding boxes in output array
output[i:i+win, j:j+2, 0:3] = [255,0,0]
output[i:i+win, j+win-2:j+win, 0:3] = [255,0,0]
output[i:i+2, j:j+win, 0:3] = [255,0,0]
output[i+win-2:i+win, j:j+win, 0:3] = [255,0,0]

# Save output image
outIm = Image.fromarray(output)
outIm.save(out_fname)

In [ ]:
# Display aircraft detected from scene_1.png
from IPython.display import Image
Image("scene_1_detection.png")